# Spoken Digit Recognition

In this part we will use HMM to perform spoken digit recognition. The task of spoken digit recognition is a simplified task of the general ASR task, where (1) only a single word needs to be recognized, and (2) the number of possible words to be recognitized is small (10-way classification). We will use this as an example for the application of HMM.

Here we will show a simple pipeline for spoken digit recognition with HMM. We will first extract MFCC features from the given waveforms for digits 0-9, and we will train 10 HMM models on each of the digits. For test data, we will pass each utterance to the 10 HMMs, calculate their generation probabilities given the models (remember that this is the evaluation problem in the 3 basic problems for HMMs), and predict its label according to the 10 probabilities.

Why do we need 10 HMMs? Recall that HMM is by define a *generative model*, however here we want to perform a *discriminative task* to assign labels to the utterances. To do so, we can train 10 different generative models for the 10 digits, and calculate the generation probability of a given utterance (sequence of MFCC features) for each of the models. The model with the highest probability can be set as the predicted label.

In this part we will not ask you to implement the scoring algorithm (Forward-Backward) or the training algorithm (EM). We will use a package named [***hmmlearn***](https://hmmlearn.readthedocs.io/en/latest/) to build and train the HMM models. If you are interested in the detailed implementation of them, you can check their [released codes in scikit-learn style](https://github.com/hmmlearn/hmmlearn).

In [1]:
# install the hmmlearn package if you don't have it installed before
!pip install hmmlearn

     |████████████████████████████████| 120 kB 8.2 MB/s eta 0:00:01


## 1. Data Preparation

Similar to the previous homework, download the data from the link provided in Courseworks and unzip it to the same folder as the Jupyter notebook scripts. The dataset we use here is the [Free Spoken Digit Dataset (FSDD)](https://github.com/Jakobovski/free-spoken-digit-dataset), which contains 3000 recordings (6 speakers, 50 of each digit per speaker). We will use 80% for training and 20% for testing (no validation set here).

In [2]:
import numpy as np
import librosa
import os
import time
import h5py

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from hmmlearn import hmm

import matplotlib.pyplot as plt
from IPython.display import Audio

In [3]:
dir_path = 'digits'  # directory path

# walk through the directory, find the files with .wav extension
wav_files = []
for (dirpath, dirnames, filenames) in os.walk(dir_path):
    for file in filenames:
        if '.wav' in file:
            wav_files.append(dirpath+'/'+file)
        
wav_files = sorted(wav_files)
num_data = len(wav_files)
print('Number of .wav files: {:2d}'.format(num_data))
print('Example path: ' + wav_files[0])

Number of .wav files: 3000
Example path: digits/0_george_0.wav


The file names have the format *{digitLabel}_{speakerName}_{index}.wav*, so we know that the first character in the file names corresponds to the digit label. Let's extract them.

In [4]:
labels = []
for i in range(num_data):
    labels.append(int(wav_files[i].split('/')[1][0]))

In [5]:
# listen to a sample

y, sr = librosa.load(wav_files[100])
Audio(y, rate=sr)

In [6]:
# split the data into training and test sets

training_set = []
testing_set = []

training_label = []
testing_label = []

rnd_seed = 20210323
np.random.seed(rnd_seed)

for digit in range(10):
    # each digit has 300 samples
    # randomly permute the utterances
    digit_idx = list(np.random.permutation(range(300)))
    digit_files = list(np.array(wav_files[digit*300:(digit+1)*300])[digit_idx])
    digit_label = list(np.array(labels[digit*300:(digit+1)*300])[digit_idx])
    
    training_set += digit_files[:240]
    testing_set += digit_files[240:]
    
    training_label += digit_label[:240]
    testing_label += digit_label[240:]

In [67]:
# TODO: extract MFCC features for each of the utterance similar to HW1
# use 30-dim MFCC features

def freq_to_mel(freq):
    return 2595.0 * np.log10(1.0 + freq / 700.0)

def met_to_freq(mels):
    return 700.0 * (10.0**(mels / 2595.0) - 1.0)

def get_filter_points(fmin, fmax, mel_filter_num, FFT_size, sample_rate=44100):
    fmin_mel = freq_to_mel(fmin)
    fmax_mel = freq_to_mel(fmax)
    
    mels = np.linspace(fmin_mel, fmax_mel, num=mel_filter_num+2)
    freqs = met_to_freq(mels)
    
    return np.floor((FFT_size + 1) / sample_rate * freqs).astype(int), freqs
def get_filters(filter_points, FFT_size):
    
    filters = np.zeros((len(filter_points)-2,int(FFT_size/2+1)))
    
    for n in range(len(filter_points)-2):
        filters[n, filter_points[n] : filter_points[n + 1]] = np.linspace(0, 1, filter_points[n + 1] - filter_points[n])
        filters[n, filter_points[n + 1] : filter_points[n + 2]] = np.linspace(1, 0, filter_points[n + 2] - filter_points[n + 1])
    
    return filters

def dct(dct_filter_num, filter_len):
    basis = np.empty((dct_filter_num,filter_len))
    basis[0, :] = 1.0 / np.sqrt(filter_len)
    
    samples = np.arange(1, 2 * filter_len, 2) * np.pi / (2.0 * filter_len)

    for i in range(1, dct_filter_num):
        basis[i, :] = np.cos(i * samples) * np.sqrt(2.0 / filter_len)
        
    return basis
MFCC_tr=[]
for nameF in training_set:
    y, sr = librosa.load(nameF)

    audio_spec = librosa.stft(y, n_fft=512, hop_length=256)
    audio_spec_mag = np.abs(audio_spec)
    # TODO: calculate MFCC using the magnitude spectrogram above
    # use 64 filters for Mel filterbank
    # name your output as mel_spec

    ## reference: https://www.kaggle.com/ilyamich/mfcc-implementation-and-tutorial
    freq_min = 0
    freq_high = sr / 2
    mel_filter_num=64
    FFT_size=512


    filter_points, mel_freqs = get_filter_points(freq_min, freq_high, mel_filter_num, FFT_size, sample_rate=sr)


    filters = get_filters(filter_points, FFT_size)


    # taken from the librosa library This is being done to prevent the noise increase because if the increase of bandwidth
    enorm = 2.0 / (mel_freqs[2:mel_filter_num+2] - mel_freqs[:mel_filter_num])
    filters *= enorm[:, np.newaxis]

    audio_filtered = np.dot(filters, (audio_spec_mag)**2)+1e-8
    mel_logP = 10.0 * np.log10(audio_filtered)

    dct_filter_num = 30 # reduce the dimenssion from  filterbank=64 to 26 DCTs and then only keep 12 lower freqs

    dct_filters = dct(dct_filter_num, mel_filter_num)

    cepstral_coefficents = np.dot(dct_filters, mel_logP)
    mel_spec=cepstral_coefficents[:30, :]
    MFCC_tr.append(mel_spec)
MFCC_ts=[]    
for nameF in testing_set:
    y, sr = librosa.load(nameF)

    audio_spec = librosa.stft(y, n_fft=512, hop_length=256)
    audio_spec_mag = np.abs(audio_spec)
    # TODO: calculate MFCC using the magnitude spectrogram above
    # use 64 filters for Mel filterbank
    # name your output as mel_spec

    ## reference: https://www.kaggle.com/ilyamich/mfcc-implementation-and-tutorial
    freq_min = 0
    freq_high = sr / 2
    mel_filter_num=64
    FFT_size=512


    filter_points, mel_freqs = get_filter_points(freq_min, freq_high, mel_filter_num, FFT_size, sample_rate=sr)


    filters = get_filters(filter_points, FFT_size)


    # taken from the librosa library This is being done to prevent the noise increase because if the increase of bandwidth
    enorm = 2.0 / (mel_freqs[2:mel_filter_num+2] - mel_freqs[:mel_filter_num])
    filters *= enorm[:, np.newaxis]

    audio_filtered = np.dot(filters, (audio_spec_mag)**2)+1e-8
    mel_logP = 10.0 * np.log10(audio_filtered)

    dct_filter_num = 30 # reduce the dimenssion from  filterbank=64 to 26 DCTs and then only keep 12 lower freqs

    dct_filters = dct(dct_filter_num, mel_filter_num)

    cepstral_coefficents = np.dot(dct_filters, mel_logP)
    mel_spec=cepstral_coefficents[:30, :]
    MFCC_ts.append(mel_spec)
    

## 2. Model Training

We use an [HMM with Gaussian Models](https://hmmlearn.readthedocs.io/en/latest/api.html#hmmlearn-hmm) for our models.

In [70]:
help(hmm.GaussianHMM)
HMM_model = hmm.GaussianHMM(n_components=5)

Help on class GaussianHMM in module hmmlearn.hmm:

class GaussianHMM(hmmlearn.base._BaseHMM)
 |  GaussianHMM(n_components=1, covariance_type='diag', min_covar=0.001, startprob_prior=1.0, transmat_prior=1.0, means_prior=0, means_weight=0, covars_prior=0.01, covars_weight=1, algorithm='viterbi', random_state=None, n_iter=10, tol=0.01, verbose=False, params='stmc', init_params='stmc')
 |  
 |  Hidden Markov Model with Gaussian emissions.
 |  
 |  Parameters
 |  ----------
 |  n_components : int
 |      Number of states.
 |  
 |  covariance_type : string, optional
 |      String describing the type of covariance parameters to
 |      use.  Must be one of
 |  
 |      * "spherical" --- each state uses a single variance value that
 |        applies to all features.
 |      * "diag" --- each state uses a diagonal covariance matrix.
 |      * "full" --- each state uses a full (i.e. unrestricted)
 |        covariance matrix.
 |      * "tied" --- all states use **the same** full covariance matri

The *n_components* parameter defines the number of states in the HMM. We typically define the number of states in a heuristic way, and here we set it to 5. For other parameters we can simply use the default configurations.

For each of the digit, we build an HMM and train it with the extracted MFCC features. Note that the length for different utterances and their MFCC features might have different length, so we need to explicitly determine the length for each of the training sequence when train the HMM models. The *fit* function allows you to explicitly determine the length of different input sequences during model training, and please take a look at the annotation.

In [71]:
# TODO: build and train 10 HMM models for the 10 digits
HMMs=[]
for i in range(10):
    D=[np.array(item).T for idd,item in enumerate(MFCC_tr) if training_label[idd]==i]
    lengths=[idd.shape[0] for idd in D ]
    F=np.concatenate(D,axis=0)
    HMMs.append(hmm.GaussianHMM(n_components=5).fit(F, lengths=lengths))

In [72]:
print(MFCC_ts[1].T.shape)

(40, 30)


## 3. Scoring and Label Prediction

Now we have 10 HMM models trained for the 10 digits. For evaluation, we pass each utterance in the test set to the 10 HMM models, calculate their generation probability (scoring), and select the one with the highest score as its label.

You need to achieve 90% accuracy to get full score.

In [73]:
# TODO: calculate the score for each of the test utterance and predict its digit label

pred=[]
for tst in MFCC_ts:
    scores=[item.score(tst.T, lengths=[tst.shape[1]]) for item in HMMs]
    pred.append(np.argmax(scores))
    

In [75]:
# TODO: calculate the accuracy
print(len(testing_label))
acc=[item for i,item in enumerate(pred) if item==testing_label[i]]
print('accouracy is: ', len(acc)/len(testing_label)*100)

600
accouracy is:  88.5


You can select the misclassified utterances, listen to it, compare the actual label and the predicted label, and think about why the model made wrong predictions.

In [77]:
#bad samples:
missed=[i for i,item in enumerate(pred) if item!=testing_label[i]]
y, sr = librosa.load(testing_set[missed[1]])
Audio(y, rate=sr)

In [78]:
# TODO: enter your discussions here

#The samples that were missed did not sound articulated well enogh ( shero instead of zero or etc. ) but disregarding these
# the model is doing a good job in prediction.


## Discussion: HMM in Harder Tasks

The use of HMM is pretty straightforward and simple here: we treat each frame of MFCC feature as an observation in a sequence. In more practical applications such as a full ASR system, HMM can be used in both the acoustic model and the language model to model the relationship between the phonemes, characters or the words. There are several challenges in those systems compared with the one we use here:
- The number of possible states as well as the search space can be significantly more complex than the digit recognition task. In this case, we need to use a much larger or complicated model and a much longer training time.
- There might be multiple HMM systems for different parts in the full system. For example, a phoneme-level HMM is needed for the acoustic modeling part, and a word-level HMM is needed for the language modeling part.
- The digit dataset we are using here does not contain noise. How to ensure that the HMM models to be noise robust is an important question for real-world applications.

We will not cover these topics here. If you are interested in more about HMM in ASR (in traditional ASR systems), you can take a look at [this great note](https://mi.eng.cam.ac.uk/~mjfg/mjfg_NOW.pdf).